***
# Easy Cash - Credit Risk Classification
Uma das área que causa mais problemas para bancos, financiadoras e empresários é a taxa de inadimplência por parte da sua carteira de clientes. Avaliar se uma pessoa irá deixar de cumprir com suas obrigações financeiras é de extrema importância, e pode causar um grande impacto no balanço da Instituição. Cada vez mais soluções vêm sendo desenvolvidas e aprimoradas visando minimizar o ***risco de default***.


***Defaulf:*** é o termo utilizado para indicar o não cumprimento das obrigações e/ou condições de um empréstimo (como financiamentos ou dívidas de cartão de crédito)
***

## 1. BUSINESS UNDERSTANDING

#### Business Problem
A Easy Cash é um banco digital que atua desde 2015 no Brasil oferecendo produtos financeiros como: cartões de crédito, seguros e emprestímos. Uma das área que causa mais problemas para bancos, fintechs é a taxa de inadimplência por parte da sua carteira de clientes. No último resultado trimestral divulgado pela empresa, a diretoria financeira, comunicou que o número de inadimplentes após a liberação de crédito aumentou significativamente, tendo o ***EL (Expected Loss)*** com um aumento de 15% representando um ***EC (Economic Capital)*** de R$ 6.2 milhões para cobrir essas perdas inesperadas. 

Portanto, foi solicitado ao time de Data Science, que realizasse um estudo das informações financeiras e das solicitações de empréstimo para encontrar padrões que possam indicar uma possível inadimplência e determinar quais ações possam ser implementadas, afim de, reduzir estas perdas financeiras por conta de pessoas mutuárias que não quitam suas dívidas.


##### Business Objective
Assim é encomendado um estudo com os seguintes objetivos:
- Reduzir em 10% as Perdas Esperadas (EL);
- Reduzir o Capital Econômico (EC) para abaixo de R$ 2Mi; 

#### Success Criteria
Este estudo será considerado um sucesso se:
- Conseguir modelar um classificador capaz de encontrar potenciais clientes inadimplentes e implementá-lo em produção através de uma API;
- Reduzir em 15% o EL;
- Manter o EC abaixo de R$ 1Mi;

## 2. DATA UNDERSTANDING

<< Descrição aqui >>

### 2.0 IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (12,6))

### 2.1 LOADING DATA

A base de dados foi disponibilizada pelo cliente em 4 tabelas distintas (descritas abaixo) no formato de dumps (.sql), os dados das tabelas foram carregados no banco de dados ` analise_risco` no MySQL, posteriormente foi realizado o ***union data*** consolidando os dados em apenas uma tabela `dados_juntos.csv` exportado em formato .csv para consumo.

Para consultar o dicionário de dados , consulte o readme: `/data/readme.md`

In [2]:
dados = pd.read_csv('../data/raw/dados_unidos.csv')
dados.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,21.0,60000.0,Rent,2.0,Education,B,8000.0,11.26,0.0,0.13,N,3.0
1,40.0,70000.0,Mortgage,6.0,Education,A,18000.0,7.90,0.0,0.26,N,14.0
2,29.0,115000.0,Mortgage,3.0,Medical,C,22000.0,15.23,0.0,0.19,Y,7.0
3,26.0,84996.0,Rent,0.0,Medical,B,9000.0,11.26,0.0,0.11,N,2.0
4,23.0,54500.0,Rent,2.0,Debtconsolidation,A,8000.0,NaN,0.0,0.15,N,2.0


In [3]:
# Verificando a qtd de registros
print(f'Número de Registros: {dados.shape[0]}')
print(f'Número de Colunas: {dados.shape[1]}')

Número de Registros: 34501
Número de Colunas: 12


### 2.2 RENAMING COLUMNS E DATA

In [4]:
# Renomeando nomes das colunas - tradução (en -> pt)
dict_col_name = {
    'person_age': 'idade_usuario',
    'person_income': 'salario_usuario',
    'person_home_ownership': 'situacao_moradia_usuario',
    'person_emp_length': 'tempo_de_trabalho_usuario',
    'loan_intent': 'motivo_emprestimo',
    'loan_grade': 'pontuacao_emprestimo',
    'loan_amnt': 'valor_emprestimo',
    'loan_int_rate': 'taxa_juros_emprestimo',
    'loan_status': 'status_emprestimo',
    'loan_percent_income': 'renda_percentual_emprestimo',
    'cb_person_default_on_file': 'devendo',
    'cb_person_cred_hist_length': 'tempo_de_credito'
}
# axis=1 -> coluna
dados.rename(dict_col_name, axis= 1, inplace=True)

In [5]:
# Renomeando dados das linhas - tradução (en -> pt)
dict_situacao_moradia = {
    'Rent': 'Alugada',
    'Own': 'Propria',
    'Mortgage': 'Hipotecada',
    'Other': 'Outros'
}

dict_motivo_emprestimo = {
    'Personal':'Pessoal',
    'Education': 'Educativo',
    'Medical':'Medico',
    'Venture':'Empreendimento',
    'Homeimprovement': 'Melhoria do Lar',
    'Debtconsolidation': 'Pagamento de Debitos'
}
# axis=0 (default) -> linhas
dados.replace(dict_situacao_moradia, inplace=True)
dados.replace(dict_motivo_emprestimo, inplace=True)

### 2.3 CLEANING DATA


Sabemos que boa parte do tempo e sucesso de um projeto de Machine Learning concentrasse na limpeza dos dados, os dados preparados permitem que os modelos treinem mais rapidamente, além de impactar significativamente o desempenho do modelo, desta forma será garantido a qualidade da base de dados, tratando valores ausentes, nulos e remoção de outliers.

### i) Missing Data

In [6]:
# Verificando a presença de valores nulos
dados.isnull().sum()

idade_usuario                   324
salario_usuario                 339
situacao_moradia_usuario        331
tempo_de_trabalho_usuario      1254
motivo_emprestimo               315
pontuacao_emprestimo            313
valor_emprestimo                331
taxa_juros_emprestimo          3630
status_emprestimo               343
renda_percentual_emprestimo     319
devendo                         370
tempo_de_credito                  4
dtype: int64

**NOTAS:** É possível verificar que há dados nulos/ausentes em todas as categorias sejam de dados númericos ou categóricos, que precisam ser tratados. 

##### DROPNA( ) - Removendo dados ausentes
a) `status_emprestimo`: Os dados ausentes na nossa variável default `status_emprestimo` não poderá ser utilizado para prever a probabilidade de inadimplência, porque não saberemos se o empréstimo foi inadimplente ou não, além de se tratar da resposta que procuramos da variável dependente.

b) `taxa_juros_emprestimo`: De forma semelhante ao item anterior, ter dados ausentes neste campo dificultará as previsões do modelo. Como as taxas de juros são definidas pela empresa, e temos valores ausentes nela, é provável que tenha ocorrido alguns erros durante a ingestão de dados.

Portanto, afim de garantir a qualidade dos dados, o time entendeu que seria melhor remover a presença de dados nulos nestes campos.

In [8]:
# Deletando valores ausentes
dados_tratados = dados.dropna(subset = ['status_emprestimo', 'taxa_juros_emprestimo'])
dados_tratados.isnull().sum()

idade_usuario                   235
salario_usuario                 245
situacao_moradia_usuario        229
tempo_de_trabalho_usuario      1079
motivo_emprestimo               241
pontuacao_emprestimo            233
valor_emprestimo                232
taxa_juros_emprestimo             0
status_emprestimo                 0
renda_percentual_emprestimo     220
devendo                         265
tempo_de_credito                  4
dtype: int64

##### FILLNA( ) - Preenchendo dados ausentes
`tempo_de_trabalho_usuario`: A falta de dados neste campo não seria tão prejudicial, mas ainda assim, poderia causar problemas nos dados de treinamento do modelo.



Portanto, juntamente com o time técnico alinhado com a área de negócios, foi decidido que estes valores ausentes fossem substituidos pelo valor da mediana.

In [11]:
# Preenchendo Valores Ausentes
dados_tratados['tempo_de_trabalho_usuario'].fillna((dados['tempo_de_trabalho_usuario'].median()), inplace= True )
dados_tratados.isnull().sum()

C:\Users\reyna\AppData\Local\Temp\ipykernel_88972\2117196173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_tratados['tempo_de_trabalho_usuario'].fillna((dados['tempo_de_trabalho_usuario'].median()), inplace= True )


idade_usuario                  235
salario_usuario                245
situacao_moradia_usuario       229
tempo_de_trabalho_usuario        0
motivo_emprestimo              241
pontuacao_emprestimo           233
valor_emprestimo               232
taxa_juros_emprestimo            0
status_emprestimo                0
renda_percentual_emprestimo    220
devendo                        265
tempo_de_credito                 4
dtype: int64

In [18]:
dados_tratados[dados_tratados['idade_usuario'].isnull()].head(100)

,idade_usuario,salario_usuario,situacao_moradia_usuario,tempo_de_trabalho_usuario,motivo_emprestimo,pontuacao_emprestimo,valor_emprestimo,taxa_juros_emprestimo,status_emprestimo,renda_percentual_emprestimo,devendo,tempo_de_credito
10,NaN,45000.0,Propria,8.0,Medico,B,NaN,6.03,0.0,0.16,N,4.0
136,NaN,70000.0,Alugada,5.0,Educativo,B,13225.0,16.11,0.0,NaN,Y,7.0
180,NaN,41160.0,Alugada,2.0,Pagamento de Debitos,A,NaN,12.99,0.0,0.29,N,3.0
503,NaN,113000.0,NaN,7.0,Educativo,A,3600.0,9.99,0.0,0.10,N,4.0
556,NaN,90000.0,Hipotecada,3.0,Educativo,A,6000.0,7.68,0.0,0.09,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14955,NaN,60000.0,Alugada,3.0,Medico,C,5000.0,14.11,1.0,0.15,N,4.0
14966,NaN,82000.0,NaN,5.0,Educativo,C,5000.0,12.69,1.0,0.17,N,4.0
15026,NaN,80000.0,Alugada,4.0,Pagamento de Debitos,A,4000.0,14.22,0.0,NaN,Y,10.0
15112,NaN,110000.0,Alugada,9.0,Pagamento de Debitos,B,12000.0,11.11,0.0,NaN,N,5.0


In [14]:
dados_tratados.describe()

,idade_usuario,salario_usuario,tempo_de_trabalho_usuario,valor_emprestimo,taxa_juros_emprestimo,status_emprestimo,renda_percentual_emprestimo,tempo_de_credito
count,30363.000000,3.035300e+04,30598.000000,30366.000000,30598.000000,30598.000000,30378.000000,30594.000000
mean,27.710042,6.598101e+04,4.760213,9598.059507,11.015236,0.219263,0.170217,5.790449
std,6.293575,6.149278e+04,4.078501,6319.876300,3.241917,0.413754,0.106978,4.038053
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.845000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,7.900000e+04,7.000000,12250.000000,13.470000,0.000000,0.230000,8.000000
max,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [19]:
dados_tratados['situacao_moradia_usuario'].unique()

array(['Alugada', 'Hipotecada', 'Propria', 'Outros', nan], dtype=object)

In [20]:
dados_tratados[dados_tratados['situacao_moradia_usuario'].isnull()].head()

,idade_usuario,salario_usuario,situacao_moradia_usuario,tempo_de_trabalho_usuario,motivo_emprestimo,pontuacao_emprestimo,valor_emprestimo,taxa_juros_emprestimo,status_emprestimo,renda_percentual_emprestimo,devendo,tempo_de_credito
239,36.0,32000.0,NaN,9.0,Educativo,B,NaN,7.51,0.0,0.16,N,5.0
269,33.0,NaN,NaN,4.0,Empreendimento,A,7000.0,12.98,0.0,0.07,Y,8.0
411,31.0,50000.0,NaN,3.0,NaN,A,4500.0,5.79,0.0,0.12,N,2.0
465,35.0,88000.0,NaN,18.0,Medico,B,4000.0,10.99,0.0,0.04,NaN,9.0
503,NaN,113000.0,NaN,7.0,Educativo,A,3600.0,9.99,0.0,0.10,N,4.0


Parei aqui, esta versão. Surgiu a dúvida que a melhor forma para garantir a qualidade dos dados é removendo todos os dados nulos, devido a falta de alinhamento com o time de negócios.